In [1]:
%load_ext autoreload
%autoreload 2

from kaggle_speech_recog import *
import math

# Get feature and label matrices

In [2]:
X_vector_len = 16000
train = SpeechList.get_train('train/audio')
XY_train_valid = train.get_spectrogram_X_and_Y(X_vector_len=X_vector_len, take_log=True, split=0.85)

SKIPPED train/audio/_background_noise_/README.md File format b'# Ba'... not understood.


/usr/local/anaconda3/lib/python3.5/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


# Train
* Hmm, sparse matrix doesnt work

In [3]:
# Need this while testing graph so it gets autoloaded
from kaggle_speech_recog.graphs.useful_tf_graph import *
from kaggle_speech_recog.graphs.test_spectrogram_conv import *

In [13]:
graph_name = 'TestSpectrogramConv'  # Specify graph class name

g_cnfg = Config('trial_graph')  # Specify graph configuration name
g_cnfg.X_img_h = XY_train_valid[0].shape[1]
g_cnfg.X_img_w = XY_train_valid[0].shape[2]
g_cnfg.Y_vector_len = XY_train_valid[1].shape[1]

g_cnfg.conv1_n_filters = 16
g_cnfg.conv1_kernel_size = 5
g_cnfg.conv1_mp_size = 5
g_cnfg.conv1_mp_strides = 2

g_cnfg.n_hidden = 50
g_cnfg.lr_initial = 0.1
g_cnfg.lr_decay_steps = 1000000
g_cnfg.lr_decay_rate = 0.7  # 1.0 means no decay

t_cnfg = Config('trial_run_01')  # Specify train configuration name
t_cnfg.ckp_dir = 'checkpoints'
t_cnfg.tb_dir = 'tensorboard'
t_cnfg.log_dir = 'logs'
t_cnfg.le = train.le
t_cnfg.max_step = 100000000
t_cnfg.batch_size = 150
t_cnfg.dropout_keep_prob = 0.5
t_cnfg.log_every = 50
t_cnfg.print_every = math.ceil(XY_train_valid[0].shape[0]/t_cnfg.batch_size)  # every epoch
t_cnfg.n_ave_ll_valid = 8
t_cnfg.start_step_early_stopping = 100000000
t_cnfg.early_stopping_patience =0.1

In [14]:
print(g_cnfg)
print(t_cnfg)

GraphClass = globals()[graph_name]
graph = GraphClass(g_cnfg)
graph.train_model(t_cnfg, XY_train_valid)

name: trial_graph
X_img_h: 129
X_img_w: 71
Y_vector_len: 31
conv1_kernel_size: 5
conv1_mp_size: 5
conv1_mp_strides: 2
conv1_n_filters: 16
lr_decay_rate: 0.7
lr_decay_steps: 1,000,000
lr_initial: 0.1
n_hidden: 50

name: trial_run_01
batch_size: 150
ckp_dir: checkpoints
dropout_keep_prob: 0.5
early_stopping_patience: 0.1
le: LabelEncoder()
log_dir: logs
log_every: 50
max_step: 100,000,000
n_ave_ll_valid: 8
print_every: 367
start_step_early_stopping: 100,000,000
tb_dir: tensorboard

TestSpectrogramConv_trial_graph_trial_run_01
Training starts @ 12/09/2017 14:47:05
Epoch 1 Step 1 Best model saved @ 12/09/2017 14:47:06 [Ave valid] 3.481
Epoch 1 Step 1 ends @ 12/09/2017 14:47:06 [Train] 3.347, 7.3% [Valid] 3.0% [Ave valid] 3.481
Epoch 1 Step 250 Best model saved @ 12/09/2017 14:47:37 [Ave valid] 3.416
Epoch 1 Step 300 Best model saved @ 12/09/2017 14:47:43 [Ave valid] 3.291
Epoch 1 Step 350 Best model saved @ 12/09/2017 14:47:49 [Ave valid] 3.142
Epoch 1 Step 367 Best model saved @ 12/09/201

Epoch 24 Step 8,808 ends @ 12/09/2017 15:05:42 [Train] 3.417, 3.4% [Valid] 3.2% [Ave valid] 3.401
Epoch 25 Step 9,175 ends @ 12/09/2017 15:06:29 [Train] 3.396, 2.6% [Valid] 3.2% [Ave valid] 3.400


KeyboardInterrupt: 